The NHANES survey includes a substantive nutritional component that asks individuals to record what they ate over two days. This yielded a set of ~120 features corresponding to the amount of various nutrients consumed across each of these days. 

Unsupervised approach –  
1. Limited sample to adults who completed both days of the survey, and who reported both days as reflecting their usual food consumption patterns. 
2. Summed nutrientional intake across the two days for each individual (reducing our feature space to ~ 60). 
3. Used PCA to reduce set of 60 nutrients to a lower-dimensional feature space. 
4. Used this reduced set of components to cluster individuals based on their nutritional profile. 

Supervised extension –

There's a vast literature on the social determinants of health that explores how demographic factors (e.g., race, income, etc.) affect health outcomes. We wanted to see how nutritional intake plays into this (specifically given the increasing attention given to disparities in food access / food sarcity). Specifically, we ran models predicting obesity and high blood pressure using just demographic factors. We then also included the nutritional profiles gleaned from above to see how this affects predictive performance. Specifically we were interested in how meaningful the clusters were as predictors, and how meaningful the components were as predictors. 


Other thought – 

Maybe instead of supervised ML, we should use regression analysis (to see if the cluster assignment / components are statistically significant predictors after controlling for demographic variables)? Effectively, this would be how to disentangle the issue of demographic factors affecting the outcome, but also affecting the clusters (which you mentioned earlier). Given that the nutritional features don't seem to add much, I wonder if this is what's going on anyways. 

In [1]:
import pandas as pd 
import numpy as np 

from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

pd.set_option('display.max_columns', 500)

In [2]:
df = pd.read_csv('clustered_data.csv')
df.head()

,year,SEQN,BMXBMI,BPQ020,RIDAGEYR,RIAGENDR,INDFMPIR,RIDRETH1,TKCAL,TPROT,TCARB,TSUGR,TTFAT,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18,PC19,PC20,assignment_kmeans
0,2007-2008,41475,58.04,1,62,2,1.83,5,3057,139.31,348.69,160.48,125.33,-1.617823,1.919746,0.455475,-0.172202,0.297086,2.514255,-0.836439,-1.447478,1.210543,0.014087,3.585609,-1.442924,1.064670,-0.591428,1.113903,1.035296,-0.094012,0.249038,0.889505,-1.001357,1
1,2007-2008,41479,27.56,2,52,1,2.20,1,4525,215.80,531.05,197.91,179.99,1.190533,-1.640098,-2.723926,0.382880,0.345203,-0.298905,-0.327906,0.688157,-1.332718,0.432461,-1.024977,0.579877,1.673101,-0.458015,-0.004225,0.127652,-0.558152,-0.262917,-1.027508,-0.741834,1
2,2007-2008,41483,44.06,1,66,1,1.14,4,5620,205.34,758.05,207.17,219.10,4.022577,-1.276317,0.697245,1.035136,-2.475559,-1.047524,-0.086807,-0.030414,-0.990138,1.059824,2.126264,-1.084899,0.479882,-0.887813,-0.532492,-0.129027,-1.944706,-0.377889,-2.218713,-0.895268,2
3,2007-2008,41485,25.99,2,30,2,1.01,2,3118,111.14,315.96,127.14,157.96,-3.524582,2.625010,-0.403105,0.151470,-0.711641,0.225901,0.996899,0.639086,-0.611923,-0.044899,-0.729598,0.503148,-0.383034,0.470820,0.336835,0.293946,-0.393011,-0.438882,-0.431600,-0.221301,1
4,2007-2008,41486,31.21,1,61,2,1.75,1,3171,87.98,558.07,279.28,78.93,-3.841056,-1.749876,1.175520,1.888064,0.689904,-0.950871,-0.905093,-0.764330,-0.566994,-1.560574,0.875038,-1.047852,-0.334103,0.376641,0.124650,-0.660536,-0.928691,0.317645,0.210854,-0.336989,1


In [3]:
# process data 
df['is_obese'] = np.where(df['BMXBMI'] > 30, 1, 0)
df['has_high_bp'] = np.where(df['BPQ020'] == 1, 1, 0)
df['male'] = np.where(df['RIAGENDR'] == 1, 1, 0)
df['white'] = np.where(df['RIDRETH1'] == 3, 1, 0)
df['black'] = np.where(df['RIDRETH1'] == 4, 1, 0)
df['hispanic'] = np.where(df['RIDRETH1'] < 3, 1, 0)

In [4]:
def run_model(train_df, test_df, label, feat, t): 
    train_X, test_X = train_df[feat], test_df[feat]
    train_y, test_y = train_df[label].ravel(), test_df[label].ravel()
    model = RandomForestClassifier(n_estimators = 100, random_state = 123)

    clf = model.fit(train_X, train_y)
    test_prob = clf.predict_proba(test_X)[:,1]
    pred = np.where(test_prob > t, 1, 0)
    
    metrics = (accuracy_score(test_y, pred), 
               precision_score(test_y, pred), 
               recall_score(test_y, pred)) 

    feat_imp = pd.DataFrame(clf.feature_importances_,
                            index = train_X.columns,
                            columns=['Importance']).sort_values('Importance', ascending = False)
    
    return metrics, feat_imp

In [5]:
# just demographic features 
train_df, test_df = train_test_split(df)
label = 'is_obese'
feat = ['male', 'RIDAGEYR', 'white', 'black', 'hispanic', 'INDFMPIR']

metrics, feat_imp = run_model(train_df, test_df, label, feat, 0.7)
print(metrics)
print(feat_imp)

(0.6308805790108565, 0.4597156398104265, 0.16302521008403362)
          Importance
INDFMPIR    0.608746
RIDAGEYR    0.341542
male        0.014293
black       0.013071
white       0.012282
hispanic    0.010066


In [6]:
# demographic features and cluster 
train_df, test_df = train_test_split(df)
label = 'is_obese'
feat = ['male', 'RIDAGEYR', 'white', 'black', 'hispanic', 'assignment_kmeans']
t = 0.7

metrics, feat_imp = run_model(train_df, test_df, label, feat, 0.7)
print(metrics)
print(feat_imp)

(0.6387213510253317, 0.49473684210526314, 0.03926482873851295)
                   Importance
RIDAGEYR             0.800308
black                0.054521
white                0.044602
hispanic             0.040985
male                 0.034640
assignment_kmeans    0.024943


In [7]:
# demographic features and components 
train_df, test_df = train_test_split(df)
label = 'is_obese'
feat = ['male', 'RIDAGEYR', 'white', 'black', 'hispanic', 'INDFMPIR', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5']
t = 0.7

metrics, feat_imp = run_model(train_df, test_df, label, feat, 0.7)
print(metrics)
print(feat_imp)

(0.6429433051869723, 0.5454545454545454, 0.005063291139240506)
          Importance
PC2         0.151047
PC5         0.140033
PC1         0.139400
PC3         0.138091
PC4         0.136697
RIDAGEYR    0.122246
INDFMPIR    0.119378
male        0.016415
white       0.012801
hispanic    0.012375
black       0.011518
